In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional.classification import f1_score, confusion_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Data Loading

In [2]:
import helper
data = helper.load_PAMAP2_acceleration()

from dataset import TimeSeriesDataset
dtset = TimeSeriesDataset(data)

weight = helper.dist(dtset, 7)

train, val = helper.split(dtset, val_pc = 0.25)
train_weight = helper.dist(train, 7)
val_weight = helper.dist(val, 7)
print(train_weight, val_weight)
train_iter = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True, num_workers = 8, pin_memory = True)
val_iter = torch.utils.data.DataLoader(val, batch_size = 32, num_workers = 8, pin_memory = True)

File exists. Loading
Keep only acceleration
Windowing
Done!
[[0.15615227]
 [0.15173083]
 [0.15151515]
 [0.18742586]
 [0.07678206]
 [0.12973148]
 [0.14666235]] [[0.15242718]
 [0.14110032]
 [0.15566343]
 [0.17961165]
 [0.0776699 ]
 [0.13786408]
 [0.15566343]]


## Model

In [3]:
from PAMAP2_model_acc import DeepConvNet

class Net(pl.LightningModule):
    def __init__(self, in_channels = 3,hidden_size = 128, output_size = 7, classes_weight = None):
        super(Net,self).__init__()
        self.model = DeepConvNet(in_channels, hidden_size, output_size)
        self.criterion = nn.CrossEntropyLoss(weight = classes_weight);
        
        self.save_hyperparameters()
        
    def forward(self, input_seq):
        return self.model(input_seq)
    
    def configure_optimizers(self):
        return torch.optim.Adamax(self.parameters(), lr=0.0001)
    
    def training_step(self, batch, batch_idx):
        y_pred = self(batch["data"])
        loss = self.criterion(y_pred, batch["label"])
        self.log('train_loss', loss, on_step = False, on_epoch = True, prog_bar = False, logger = True)
        self.log('train_f1_score', f1_score(y_pred, batch["label"], class_reduction = "micro"), on_step = False, on_epoch = True, prog_bar = True, logger = True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        y_pred = self(batch["data"])
        loss = self.criterion(y_pred, batch["label"])
        self.log('val_loss', loss, on_step = False, on_epoch = True, prog_bar = False, logger = True)
        self.log('val_f1_score', f1_score(y_pred, batch["label"], class_reduction = "micro"), on_step = False, on_epoch = True, prog_bar = True, logger = True)
        return loss

## Training

In [4]:
model = Net(hidden_size = 512, classes_weight = torch.tensor(train_weight, dtype = torch.float))

trainer = pl.Trainer(gpus=-1,
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     log_every_n_steps = 200,
                     callbacks = [EarlyStopping(monitor = "val_f1_score", patience = 5, mode = "max", verbose = True)],
                     logger = TensorBoardLogger(save_dir = 'Validation_transformer_logs/', name = "PAMAP2_acceleration"),
                     )
trainer.fit(model, train_iter, val_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepConvNet      | 6 M   
1 | criterion | CrossEntropyLoss | 0     


1

## Validation

In [5]:
# val_model = DeepConvNet(hidden_size = 256)
# state_dict = torch.load("logs/PAMAP2_LSTM_model/version_2/checkpoints/epoch=8.ckpt")["state_dict"]
# state_dict = helper.remove_prefix_from_dict("model.", state_dict)
# val_model.load_state_dict(state_dict, strict = False)
# val_model.eval()
# crit = nn.CrossEntropyLoss()

# for batch in val_iter:
#     data = batch["data"]
#     label = batch["label"]
#     output = val_model(data)
#     val_loss = crit(output, label)
#     print("Loss : ", val_loss, "\t\t F1 score : ", f1_score(output, label, class_reduction = "micro"), "\n")

## Saved Model Score
Location : logs/PAMAP2_LSTM/version_2/checkpoints/epoch=8.ckpt

F1 Score : 96.21%

Loss     : 0.1168

## Parameters
LR           : 0.0001

Optimizer    : Adamax

Epochs       : 8

LR Scheduler : None

hidden_size : 256